In [ ]:
from __future__ import absolute_import, division
from BashColors import C
from CV2_Utils_2 import *
from TarfileFunctions import *

import cv2, os
import numpy as np
from os.path import *
# import PIL
# from PIL import Image
import matplotlib.pyplot as plt
import glob, json

contentPath=os.getcwd()
planetsPath=join(contentPath, 'planets')
if not exists(planetsPath):
    tff.extractTarfiles('OriginalPlanets.tar.gz')

imagePath=join(contentPath, '3b7d7d8a64.jpg')
savePath=join(contentPath, 'tmp')
if not exists(savePath):
    os.makedirs(savePath)
    
initialGlobList:list
with open("initialGlobList.json", 'r') as f:
    initialGlobList = json.load(f)
    
def listNewFiles(initial=initialGlobList, delete=False):
    currentFilesGlob=glob.glob('**', recursive=True)
    print(f'\nNew files...')
    if len(initial) == len(currentFilesGlob):
        print(f'{C.BIRed}Nothing to see here.\nNo new files.')
    for fil in currentFilesGlob:
        if not fil in initial:
            if isdir(fil):
                print(f'{C.BIBlue}{fil}')
                if delete:
                    shutil.rmtree(fil)
            elif isfile(fil):
                print(f'{C.ColorOff}{fil}')
                if delete:
                    os.remove(fil)
listNewFiles()

In [ ]:
def squareImage(image):
    if image.shape[0] != image.shape[1]:
        print(image.shape)
        return(image)
    else:
        print('image is rectangular')
        return image

In [ ]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

def warpaffineRotation(imagePath, angle):
    print(imagePath)
    image = cv2.imread(imagePath)

    (h, w) = image.shape[:2]
    center = (w / 2, h / 2)
    angle = angle
    scale = 1

    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

In [ ]:
def crop_square(img, size, interpolation=cv2.INTER_AREA):
    # print(f'initial shape: {img.shape}')
    h, w = img.shape[:2]
    min_size = np.amin([h,w])
    print(f'min_size: {min_size}')

    # Centralize and crop
    crop_img = img[int(h/2-min_size/2):int(h/2+min_size/2), 
                   int(w/2-min_size/2):int(w/2+min_size/2)]
    resized = cv2.resize(crop_img, (size, size), interpolation=interpolation)
    resized = cv2.resize(resized, (224, 224))
    
    cvu.plotShowTwoImages(img, resized, title1=img.shape, title2=resized.shape)

    return resized

def zoom_center(img, zoom_factor=-0.5):
    y_size = img.shape[0]
    x_size = img.shape[1]
    
    # define new boundaries
    x1 = int(0.5*x_size*(1-1/zoom_factor))
    x2 = int(x_size-0.5*x_size*(1-1/zoom_factor))
    y1 = int(0.5*y_size*(1-1/zoom_factor))
    y2 = int(y_size-0.5*y_size*(1-1/zoom_factor))

    # first crop image then scale
    img_cropped = img[y1:y2,x1:x2]
    cvu.plotShowTwoImages(img, img_cropped, title2='zoomed')
    return cv2.resize(img_cropped, None, fx=zoom_factor, fy=zoom_factor)

In [ ]:
newSavePath=None
def maintain_aspect_ratio_resize(image, width=None, height=None, inter=cv2.INTER_LINEAR_EXACT):
    # Grab the image size and initialize dimensions
    dim = None
    (h, w) = image.shape[:2]

    # Return original image if no need to resize
    if width is None and height is None:
        return image

    # We are resizing height if width is none
    if width is None:
        # Calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # We are resizing width if height is none
    else:
        # Calculate the ratio of the 0idth and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # Return the resized image
    resized=cv2.resize(image, dim, interpolation=inter)
    newSavePath=join(savePath, basename(imagePath))
    # written=cv2.imwrite(newSavePath, resized)
    if written:
        return resized
    else: return resized

In [ ]:
planetList=[]
os.chdir(planetsPath)
globList=glob.glob('**', recursive=True)
for fil in globList:
    if fil.endswith('jpeg')or fil.endswith('.png')or fil.endswith('.jpg'):
        fullPath=abspath(fil)
        planetList.append(fullPath)
        print(fullPath)
        
os.chdir(contentPath)
print(len(planetList), ' planets in planetList')

In [ ]:
import os
import glob
from PIL import Image

dst_dir = join(contentPath, 'test')
os.makedirs(dst_dir, exist_ok=True)

for pth in planetList:
    root, ext = os.path.splitext(pth)
    # print(root, ext)
    if ext in ['.jpg', '.png', '.jpeg']:
        img = Image.open(f)
        imgResize = img.resize((img.width // 2, img.height // 2))
        # savePath = os.path.savePath(root)
        savePath = root + '_half' + '.png'
        savePath = join(dst_dir, savePath)
        print(savePath)
        imgResize.save(savePath)
        # cvu.plotShowTwoImages(img, imgResize, title2='resized')
        img= None
        imgResize=None
    else: print('bullshit')
        

In [ ]:
q

In [ ]:
import cv2
tempPathList=[]
planetCount=0
for pth in planetList:
    planetCount+=1
    planetName = 'planet' + str(planetCount) + '.png'
    savePath=join(contentPath, 'tmp')
    savePath=join(savePath, planetName)
    tempPathList.append(savePath)
    
    newImage = cv2.imread(pth, cv2.IMREAD_COLOR)
    newImage = crop_square(img=newImage, size=224)
    
    if not exists(savePath):
        print(savePath)
        cv2.imwrite(filename=savePath, img=newImage)
    else: print(savePath, 'already exists')
        
    print()

In [ ]:
# scale images
for path in tempPathList:
    # print(path)
    splt=splitext(path)
    # print(splt)
    savePath=splt[0] + 'z50'
    savePath=savePath + splt[1]
    print(savePath)
    
    img=cv2.imread(path, cv2.IMREAD_COLOR)
    zoomImg=zoom_center(img)
    
    img=None
    zoomImg=None

In [ ]:
for path in tempPathList:
    splt=splitext(path)
    print(splt)
    
    
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    print(type(img))
    # cvu.plotShowSingleImage(img, title1=basename(path))
    

    img_flip_vert = cv2.flip(img, 0)
    cvu.plotShowTwoImages(img, img_flip_vert, title2='vertical')
    # cv2.imwrite('data/dst/lena_cv_flip_ud.jpg', img_flip_ud)
    # True

    img_flip_horiz = cv2.flip(img, 1)
    cvu.plotShowTwoImages(img, img_flip_horiz, title2='horizontal')
    # cv2.imwrite('data/dst/lena_cv_flip_lr.jpg', img_flip_lr)
    # True

    img_flip_both = cv2.flip(img, -1)
    cvu.plotShowTwoImages(img, img_flip_both, title2='both axis')
    # cv2.imwrite('data/dst/lena_cv_flip_ud_lr.jpg', img_flip_ud_lr)
    # True
    img=None
    img_flip_vert=None
    img_flip_horiz=None
    img_flip_both=None

In [ ]:
for path in tempPathList:
    rotatedImage = warpaffineRotation(imagePath=path, angle=22.5)
    cvu.plotShowSingleImage(rotatedImage, title1=basename(path))

In [ ]:
np.angle

In [ ]:
import numpy as np
from PIL import Image

img = np.array(Image.open('data/src/lena.jpg'))
print(type(img))
# <class 'numpy.ndarray'>

print(img.shape)
# (225, 400, 3)

Image.fromarray(np.rot90(img)).save('data/dst/lena_np_rot90.jpg')

Image.fromarray(np.rot90(img, 2)).save('data/dst/lena_np_rot90_180.jpg')

Image.fromarray(np.rot90(img, 3)).save('data/dst/lena_np_rot90_270.jpg')

In [ ]:
q
print('Original Dimensions : ',img.shape)

scale_percent = 200 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
 
# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

print('Resized Dimensions : ', resized.shape)

cvu.plotShowTwoImages(img, resized, title2=resized.shape)

In [ ]:
def aspect_ratio_resize(image, width=224, height=None, inter=cv2.INTER_LINEAR_EXACT):
    # Grab the image size and initialize dimensions
    dim = None
    (h, w) = image.shape[:2]

    # Return original image if no need to resize
    if width is None and height is None:
        return image

    # We are resizing height if width is none
    if width is None:
        # Calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # We are resizing width if height is none
    else:
        # Calculate the ratio of the 0idth and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # Return the resized image
    resized=cv2.resize(image, dim, interpolation=inter)
    newSavePath=join(savePath, basename(imagePath))
    written=cv2.imwrite(newSavePath, resized)
    if written:
        return resized

In [ ]:
img=cv2.imread(newSavePath)
resizeImg=np.copy(img)
height = resizeImg.shape[0]
width = resizeImg.shape[1]
print(height, 'width:', width)

new_height = 224
ratio = new_height / height # (or new_height / height)
print(ratio)
new_height = int(height * ratio)
print(new_height)

dimensions = (width, new_height)
resizeImg = cv2.resize(img, (224, 224),interpolation=cv2.INTER_AREA)


cvu.plotShowTwoImages(img, resizeImg, title2=resizeImg.shape)